In [ ]:
# 按台站的年份来记录，合成大文件
import numpy as np
import h5py
import os
import ipdb

# 假设所有 npz 文件都位于该文件夹中
input_directory = 'G:/云南巧家宽频带数据/denoised_mseed/QJ042022/results'
output_hdf5 = 'G:/云南巧家宽频带数据/denoised_h5/QJ042022_denoised.hdf5'

# 定义每个 npz 文件的时长（单位：秒）和合并后的时长
segment_duration = 30  # 每个 npz 文件对应的时间（30秒）
combined_duration = 200  # 合并后的时间（200秒）
needed_samples = combined_duration*100  # 当前需要的总时长（初始为 200s）


# 获取所有 npz 文件的路径列表
npz_files = sorted([f for f in os.listdir(input_directory) if f.endswith('.npz')])

# 初始化变量
wave_counter = 0
wave_data = []  # 用于存储合并的波形数据
leftover_data = None  # 用于存储上一个文件剩余的 10 秒数据

# 打开 hdf5 文件，准备写入
with h5py.File(output_hdf5, 'w') as hdf5_file:
    for i, npz_file in enumerate(npz_files):
        # 读取 npz 文件的波形数据
        #ipdb.set_trace()
        file_path = os.path.join(input_directory, npz_file)
        npz_data = np.load(file_path)
        
        waveform = npz_data['data']
        
  
        
        # 如果剩下所需截取的时长不足一个30s，则截取所需的，将其合并并存储到 hdf5 文件中
        if wave_data:
            current_samples = (np.concatenate(wave_data, axis=0)).shape[0]
        else:
            current_samples = 0
        if needed_samples - current_samples <= segment_duration*100:
            # 需要截取的时长
            needed_waveform_part = needed_samples - current_samples
            # 截取所需的 200 秒部分
            wave_data.append(waveform[:needed_waveform_part])
            
            # 将剩余的部分（10 秒）保存，供下次使用
            leftover_data = waveform[needed_waveform_part:]
            print("Waveform shape before concatenation:", waveform.shape)
            # 合并当前 200 秒的波形数据
            combined_wave = np.concatenate(wave_data, axis=0)
            combined_wave = combined_wave.squeeze(axis=1)
            print("Combined wave shape after concatenation:", combined_wave.shape)
          
            #base_name = os.path.basename(npz_file)
            parts = npz_file.split('_')
            station_id = parts[0]
            date_time_part = parts[3]  # 提取倒数第二部分，即日期时间部分
            hour_part = parts[-3].split('.')[0]
            
            # 存储到 hdf5 文件中，以 "wave_x" 为组名
            group_name = f'earthquake/{station_id}{date_time_part}.{hour_part[0:4]}{wave_counter + 1:02}'
            hdf5_file.create_dataset(group_name, data=combined_wave)
            print(f"Saved {group_name} to {output_hdf5}")

            # 重置变量，准备处理下一个 200 秒的波形
            wave_counter += 1
            wave_data = []
        else:
            # 如果有剩余的 10 秒数据，先将其拼接到当前波形前面
            if leftover_data is not None:
                waveform = np.concatenate([leftover_data, waveform], axis=0)
                leftover_data = None# 当前波形未达到 200 秒，直接添加到 wave_data 列表中
            # 当前波形未达到 200 秒，直接添加到 wave_data 列表中
            wave_data.append(waveform)
            
    # 如果最后剩余的波形数据不足 200 秒，也保存（可选）
    # if wave_data:
    #     combined_wave = np.concatenate(wave_data, axis=0)
    #     group_name = f'wave_{wave_counter}'
    #     hdf5_file.create_dataset(group_name, data=combined_wave)
    #     print(f"Saved remaining {group_name} to {output_hdf5}")


In [ ]:
# 按具体每个月来记录，合成小文件
# 多个segment组合成一个200s的波形，每个月所有的200s波形存在h5里
import numpy as np
import h5py
import os
import ipdb

# 假设所有 npz 文件都位于该文件夹中
input_directory = 'G:/云南巧家宽频带数据/denoised_mseed/QJ01202208/results'
output_hdf5 = 'G:/云南巧家宽频带数据/denoised_h5/QJ01202208_denoised.hdf5'

# 定义每个 npz 文件的时长（单位：秒）和合并后的时长
segment_duration = 30  # 每个 npz 文件对应的时间（30秒）
combined_duration = 200  # 合并后的时间（200秒）
needed_samples = combined_duration*100  # 当前需要的总时长（初始为 200s）

# 计算每次合并需要的文件数量
# files_per_wave = combined_duration // segment_duration
# remaining_duration = needed_samples % (segment_duration*100)  # 剩余未满 30 秒的部分（这里是 20秒）


# 获取所有 npz 文件的路径列表
npz_files = sorted([f for f in os.listdir(input_directory) if f.endswith('.npz')])

# 初始化变量
wave_counter = 0
wave_data = []  # 用于存储合并的波形数据
leftover_data = None  # 用于存储上一个文件剩余的数据

# 打开 hdf5 文件，准备写入
with h5py.File(output_hdf5, 'w') as hdf5_file:
    #ipdb.set_trace()
    for i, npz_file in enumerate(npz_files):
        # 读取 npz 文件的波形数据
        #ipdb.set_trace()
        file_path = os.path.join(input_directory, npz_file)
        npz_data = np.load(file_path)
        
        waveform = npz_data['data']   # (3000,1,3)
        
        
        # 如果剩下所需截取的时长不足一个30s，则截取所需的，将其合并并存储到 hdf5 文件中
        if wave_data:
            current_samples = (np.concatenate(wave_data, axis=0)).shape[0]
        else:
            current_samples = 0
        if needed_samples - current_samples <= segment_duration*100:
            # 需要截取的时长
            
            needed_waveform_part = needed_samples - current_samples
            # 截取所需的 200 秒部分
            wave_data.append(waveform[:needed_waveform_part])
            
            # 将剩余的部分（10 秒）保存，供下次使用
            leftover_data = waveform[needed_waveform_part:]
            print("Waveform shape before concatenation:", waveform.shape)
            # 合并当前 200 秒的波形数据
            combined_wave = np.concatenate(wave_data, axis=0)
            combined_wave = combined_wave.squeeze(axis=1)
            print("Combined wave shape after concatenation:", combined_wave.shape)
            # (20000,1,3)
            #base_name = os.path.basename(npz_file)
            # parts = npz_file.split('_')
            # station_id = parts[0]
            # date_time_part = parts[3]  # 提取倒数第二部分，即日期时间部分
            # hour_part = parts[-3].split('.')[0]
            
            # 存储到 hdf5 文件中，以 "wave_x" 为组名
            group_name = f'earthquake/{wave_counter + 1:02}'
            hdf5_file.create_dataset(group_name, data=combined_wave)
            print(f"Saved {group_name} to {output_hdf5}")

            # 重置变量，准备处理下一个 200 秒的波形
            wave_counter += 1
            wave_data = []
        else:
            # 如果有剩余的 10 秒数据，先将其拼接到当前波形前面
            if leftover_data is not None:
                waveform = np.concatenate([leftover_data, waveform], axis=0)
                leftover_data = None# 当前波形未达到 200 秒，直接添加到 wave_data 列表中
            wave_data.append(waveform)
            
    # 如果最后剩余的波形数据不足 200 秒，也保存（可选）
    # if wave_data:
    #     combined_wave = np.concatenate(wave_data, axis=0)
    #     group_name = f'wave_{wave_counter}'
    #     hdf5_file.create_dataset(group_name, data=combined_wave)
    #     print(f"Saved remaining {group_name} to {output_hdf5}")
    
    
